# Ph.D. Course on "Advanced Machine Learning" August 23rd, 2024
## - Physics-Informed Machine Learning and Surrogate Modelling

This notebook is a part of exercises in the PhD Course that complement the lecture. 

To make sure your environment is setup to perform the exercises please visit

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; https://github.com/apengsigkarup/SciML

and follow the instructions for installing the environment.

If you have not already done so, please download all the data files and codes from this github repository and report any errors or issues with the materials to Allan Peter Engsig-Karup (apek@dtu.dk).

### Python modules needed

In [1]:
import torch
import platform

### Detect device

In [2]:
# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

# Check for ARM architecture
architecture = platform.machine()
if 'arm' in architecture or 'aarch' in architecture:
    print("Architecture: ARM")
else:
    print(f"Architecture: {architecture}")

# Optional: More detailed GPU check
if device == "cuda":
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Number of GPUs available: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

# Check for CPU information
cpu_info = platform.processor()
if cpu_info:
    print(f"CPU: {cpu_info}")
else:
    print("CPU information not available.")


Device: cpu
Architecture: ARM
CPU: arm


### Define Precision used in PyTorch

In [3]:
# Set default tensor type in PyTorch to float64 precision
torch.set_default_dtype(torch.float64)

## Exercise 00: Getting Started

The purpose of this notebook is to make sure that your enviroment is ready for the exercises. This is done by testing the cells below.

### 1. Define and create a simple neural network 

Let's define a class for a simple neural network using Pytorch and print the model configuration. The network is to have on hidden layer and take an input vector of dimension 50, a hidden layer with 100 neurons and an output layer of 10 nodes. All nodes should include bias terms and the hidden layer should be activated using a nonlinear ReLu function.

In [4]:
import torch
import torch.nn as nn

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

class SimpleNN(nn.Module): 
    def __init__(self):
        super(SimpleNN, self).__init__()
        # Define the hidden layer with 100 neurons and bias 
        self.hidden = nn.Linear(in_features=50, out_features=100) 
        # Define the output layer with 10 neurons and bias 
        self.output = nn.Linear(in_features=100, out_features=10)

    def forward(self, x):
        # Activation function for hidden layer 
        x = torch.tanh(self.hidden(x))
        #    Output layer
        x = self.output(x)
        return x

# Create an instance of the network 
model = SimpleNN()
print(model)
print(f'Total trainable parameters: {count_parameters(model)}')

SimpleNN(
  (hidden): Linear(in_features=50, out_features=100, bias=True)
  (output): Linear(in_features=100, out_features=10, bias=True)
)
Total trainable parameters: 6110


### 2. Prepare training of the neural network

In [12]:
model = SimpleNN()
optimizer = torch.optim.LBFGS(model.parameters(), lr=0.01)

### 3. Store and load a model

In [6]:
# Utilities
# Load model and optimizer state_dict
def load_checkpoint(checkpoint, model, optimizer):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

# Save model and optimizer state_dict
def save_checkpoint(state, filename="default_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)

In [7]:
SAVE_MODEL_FILE = "model_statedict.pth.tar"
checkpoint = {
    "state_dict": model.state_dict(),
    "optimizer": optimizer.state_dict(),
}
save_checkpoint(checkpoint, filename=SAVE_MODEL_FILE)

=> Saving checkpoint


In [8]:
LOAD_MODEL_FILE = "model_statedict.pth.tar"
load_checkpoint(torch.load(LOAD_MODEL_FILE), model, optimizer)


=> Loading checkpoint


### 4. Store a model in PyTorch

In [9]:
# Save the trained model
torch.save(model.state_dict(), 'model.pth')